In [55]:
import yfinance as yf
import pandas as pd
import math
from tqdm import tqdm as tqdm

DOWNLOAD = True

In [56]:
tickers = ['TSLA',
           'AAPL',
           'MSFT',
           'U',
           'CCL',
           'TD',
           'SPY',
           'FB',
           'V',
           'DIS',
           'CNR',
           'WEED.TO',
           'SNC.TO',
           'SHOP',
           'SU.TO',
           'CM.TO',
           'TD.TO',
           'ENB.TO',
           'APHA.TO',
           'XIU.TO', # s&p/tsx composite 60
           'AC.TO']

In [57]:
# u = yf.Ticker("U")
# udata = u.history(period='2y', interval = '1h')

In [58]:
#tdata.loc[tdata['name']=='2021-01-22']
# tdata.tail(10)

In [59]:
# datee=udata.iloc[1]
# datee

In [60]:
# import pandas as pd
# datee.name == pd.Timestamp('2020-09-18')
# true

In [61]:
raws = {}

for ticker in tqdm(tickers):
    t = yf.Ticker(ticker)
    t_data = t.history(period = '10y', interval = '1d')
    
    raws[ticker] = t_data
    

100%|██████████| 21/21 [00:06<00:00,  3.40it/s]


In [62]:
col_d = {} # dropped columns

for ticker in tqdm(tickers):
    raw = raws[ticker]
    
    col_d[ticker] = raw.drop(columns=['Dividends', 'Stock Splits'])
    
col_d['TSLA'].head()

100%|██████████| 21/21 [00:00<00:00, 508.56it/s]


,Open,High,Low,Close,Volume
Date,,,,,
2011-01-24,4.706,4.962,4.646,4.898,8225500
2011-01-25,4.930,4.978,4.804,4.936,6357500
2011-01-26,4.942,4.976,4.820,4.950,5399500
2011-01-27,4.948,5.016,4.906,4.984,4478500
2011-01-28,4.976,4.976,4.750,4.802,5242000


In [63]:
import numpy as np

In [64]:
good = {}

for ticker in tqdm(tickers):
    df = col_d[ticker]
    df.dropna(inplace=True)  # drops NaN rows
    
    new_cols={}
    
    for col in df.columns: # todo do all need normaling?
        new_cols[col] = df[[col]].values.reshape(-1)
        
    new_cols['DayOfWeek'] = [i.dayofweek/4 for i in df.index] # day of the week on [0,1]
    
    good[ticker] = pd.DataFrame(data=new_cols)

    

100%|██████████| 21/21 [00:00<00:00, 55.39it/s]


In [65]:
import pickle

#serializing for future use

with open('./data/std-stocks.pkl', 'wb') as f:
    pickle.dump(good, f, protocol=pickle.HIGHEST_PROTOCOL)

## Variables
- Open (n)
- High (n)
- Low (n)
- Close (n)
- Volume (n)
- Day of Week (ex. weekends)


In [66]:
# creating the 'answers'/future price
outData = pd.DataFrame(data={'Future':[]})

outData['Future'] = good['TSLA']['Close'].shift(-1)

print(good['TSLA']['Close'])

print(outData)

0         4.898000
1         4.936000
2         4.950000
3         4.984000
4         4.802000
           ...    
2512    826.159973
2513    844.549988
2514    850.450012
2515    844.989990
2516    846.640015
Name: Close, Length: 2517, dtype: float64
          Future
0       4.936000
1       4.950000
2       4.984000
3       4.802000
4       4.820000
...          ...
2512  844.549988
2513  850.450012
2514  844.989990
2515  846.640015
2516         NaN

[2517 rows x 1 columns]


In [67]:
with_future = {}    
    
# adding future price
for ticker in tqdm(good):
    df = pd.DataFrame(good[ticker])
    future = df['Close'].shift(-1) #shifts the values back by one
    df.insert(len(df.columns), 'Future', future)
    df.dropna(inplace=True)
    
    with_future[ticker] = df
    
    
    

100%|██████████| 21/21 [00:00<00:00, 284.91it/s]


In [68]:
with_future['TSLA']

,Open,High,Low,Close,Volume,DayOfWeek,Future
0,4.706000,4.962000,4.646000,4.898000,8225500,0.00,4.936000
1,4.930000,4.978000,4.804000,4.936000,6357500,0.25,4.950000
2,4.942000,4.976000,4.820000,4.950000,5399500,0.50,4.984000
3,4.948000,5.016000,4.906000,4.984000,4478500,0.75,4.802000
4,4.976000,4.976000,4.750000,4.802000,5242000,1.00,4.820000
...,...,...,...,...,...,...,...
2511,843.390015,863.000000,838.750000,845.000000,31266300,0.75,826.159973
2512,852.000000,859.900024,819.099976,826.159973,38777600,1.00,844.549988
2513,837.799988,850.000000,833.000000,844.549988,25367000,0.25,850.450012
2514,858.739990,859.500000,837.280029,850.450012,25665900,0.50,844.989990


In [69]:
import random
from collections import deque

#data is 2D array where each subarray is row

## BROKEN ##
def norm_seq(data, future):
    nfuture = future
    
    for col in range(len(data[0])):
        if col != 5:
            column = []

            for row in range(len(data)):
                column.append(data[row][col])

            if col == 3:
                column.append(future)

            sigma = np.std(column)
            mu = np.mean(column)
            
            # if the values are always the same, then sd will be 0
            sigma = 1 if sigma == 0 else sigma

            norm = lambda val: (val-mu)/sigma

            for row in range(len(data)):
                data[row][col] = norm(data[row][col])

            if col == 3:
                # price column
                nfuture = norm(future)
    
    return data, nfuture




# converts data frame into individual sequences 
# df the dataframe where last column is 'future value'
def to_seq(df):
    sequences = []
    
    # deque for rolling over
    day_sequence = deque(maxlen = 10) # deque for storing the days
    
    for row in df.values:
        # appends values (in list) to day_sequence, excluding future price
        day_sequence.append(row[:-1])
        
        if len(day_sequence) == 10:
            # deque fully populated with 10 days of data
            norm_data, norm_future = norm_seq(np.array(day_sequence), row[-1])
            sequences.append([norm_data, norm_future])
            
            
            
    return sequences





train_seq = {} # train sequences
val_seq = {} #  out of sample val (last 5%)

#convert into sequences
for ticker in tqdm(with_future):
    seq = to_seq(with_future[ticker])
    
    split = int(len(seq)*0.05)
    
    train = seq[:-split]  # 95%
    test = seq[-split:]  # 5%
    
    random.shuffle(train)
    random.shuffle(test)
    
    train_seq[ticker] = train
    val_seq[ticker] = test



100%|██████████| 21/21 [00:21<00:00,  1.01s/it]


In [70]:
df.values[0][-1]

3.069999933242798

In [71]:
train_seq['TSLA'][1]

[array([[ 0.6595362 ,  1.01885307,  0.51681092,  0.38750222,  2.33748912,
          1.        ],
        [ 0.06194939,  0.13360583,  0.25691427,  0.74705128, -0.53695831,
          0.        ],
        [ 0.7199585 ,  0.53920677,  0.66483148,  0.82767422, -0.71578595,
          0.25      ],
        [ 0.68742342,  0.52217207,  0.94767763,  0.81966255, -1.46807104,
          0.5       ],
        [ 0.73589297,  0.88452511,  1.03890004,  1.32994336, -0.0953767 ,
          0.75      ],
        [ 1.05792572,  0.91270048,  0.9000584 ,  0.77509404, -0.6988833 ,
          1.        ],
        [ 0.19474674,  0.11525884, -0.27320697, -0.09073   ,  0.08544639,
          0.        ],
        [-0.56617912, -0.81192402, -0.85783272, -0.66110217, -0.25143272,
          0.25      ],
        [-1.42537639, -1.10940771, -1.08445412, -0.86441218,  0.43770222,
          0.5       ],
        [-2.12587742, -2.20499043, -2.10969893, -1.74525954,  0.90587028,
          0.75      ]]),
 -1.5254237827145518]

In [72]:
with open('./data/train-sequences.pkl', 'wb') as f:
    pickle.dump(train_seq, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('./data/test-sequences.pkl', 'wb') as f:
    pickle.dump(train_seq, f, protocol=pickle.HIGHEST_PROTOCOL)

In [78]:
len(train_seq['APHA.TO'])

2150